In [ ]:
import numpy as np
import pandas as pd

In [ ]:
!dir

In [ ]:
data = pd.read_csv("ReportesDiqueUnificados.csv",encoding='latin1',sep=',')

In [ ]:
quejas_0 = data[data['FOLIOQUEJA'].notnull()]

In [ ]:
data.DATE_INI

In [ ]:
data2 = pd.read_excel('ReportesDiqueUnificados.xlsx')

In [ ]:
quejas2 = data2[data2['FOLIOQUEJA'].notnull()]

In [ ]:
quejas2['FOLIOQUEJA'] = quejas2['FOLIOQUEJA'].astype('int64')

In [ ]:
quejas2.dtypes

In [ ]:
columnas = ['SERVICE_ACCOUNT','FOLIOQUEJA','TIPO_TAREA','TIPO_PRUEBA',
 'DATE_INI','COPE','STATUS','DESCRIPTION','NEM_CODE','RADIUS_CODE','PE_CODE','PP_CODE','SBC_CODE','NEM_DESCRIPTION','PP_DESCRIPTION','PE_DESCRIPTION',
'SBC_DESCRIPTION','RADIUS_DESCRIPTION']

In [ ]:
quejas2[columnas].to_csv('DIQUE_REPORTE_QUEJAS.csv',index=False)

In [ ]:
!dir

In [ ]:
quejas2['DATE_INI']

In [ ]:
quejas2.count()

In [ ]:
!dir

In [ ]:
data3 = pd.read_csv('DIQUE_REPORTE_QUEJAS.csv', encoding='latin1')

In [ ]:
data3[data3[''].isnull()]

In [ ]:
data3[data3['SERVICE_ACCOUNT']==8676883011]

In [ ]:
!dir

# * Este es el bueno *

In [ ]:
data4 = pd.read_csv("DIQUE_REPORTE_QUEJAS_2.csv", sep='|')

In [ ]:
list(data4.columns)

In [ ]:
columnas2 = ['FOLIOQUEJA','TIPO_TAREA','TIPO_PRUEBA', 'CODRECEP',
 'DATE_INI','COPE','STATUS','DESCRIPTION','NEM_CODE','RADIUS_CODE','PE_CODE','PP_CODE','SBC_CODE','NEM_DESCRIPTION','PP_DESCRIPTION','PE_DESCRIPTION',
'SBC_DESCRIPTION','RADIUS_DESCRIPTION','LIQUIDACION','DESC_ETAPA','DISTRITO','CLAVE','EXPPBA']

In [ ]:
data5 = data4[columnas2]

In [ ]:
data5.to_csv('DIQUE_QUEJAS.csv',index=False)

In [ ]:
columnas3 = ['TIPO_TAREA','CODRECEP','NEM_CODE','RADIUS_CODE','PE_CODE','PP_CODE','SBC_CODE','LIQUIDACION']

In [ ]:
data4[columnas3].to_csv('DIQUE_QUEJAS-fragmento.csv',index=False)

In [ ]:
!dir

In [ ]:
datos_quejas_dique = pd.read_csv('DIQUE_QUEJAS.csv').drop_duplicates(subset='TELEFONO',keep='last')
datos_xml_dique = pd.read_excel('02-03-2019.xls').drop_duplicates(subset='SERVICE_ACCOUNT',keep='last')

dique_XML = datos_xml_dique[datos_xml_dique['SERVICE_ACCOUNT'].isin(datos_quejas_dique['TELEFONO'])]
dique_XML.to_excel('XML_filtrados_02-03-2019.xls', index=False)

## Convertir los XML a columnas de un DataFrame en pandas

In [2]:
import pandas as pd
import xmltodict
from flatten_json import flatten
from numba import jit, prange
import progressbar
import os

In [3]:
dir_xml = os.path.join(os.getcwd(),"dique_xml")

In [4]:
files = [i for i in os.listdir(dir_xml) if i.endswith('.xls')]

In [5]:
archivo = os.path.join(dir_xml, files[0])

In [6]:
data_dique = pd.read_excel(archivo, skiprows=2)

In [1]:
data_dique

NameError: name 'data_dique' is not defined

In [16]:
@jit(parallel=True)
def limpiar_xml(data_dique):
    dique_xmls = pd.DataFrame()
    bar = progressbar.ProgressBar(maxval=len(data_dique['XML']),
                                  widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage(), progressbar.Counter(), progressbar.CurrentTime()])
    bar.start()
    for i in prange(len(data_dique['XML'])):
        salida = dict(xmltodict.parse(data_dique['XML'][i]))
        aux0 = pd.DataFrame([flatten(salida)], index=[i])
        dique_xmls = pd.concat([dique_xmls,aux0], sort=False)
        bar.update(i+1)
    dique_final = pd.concat([data_dique,dique_xmls], sort=False, axis=1)
    bar.finish()
    return dique_final

In [17]:
dique_final = limpiar_xml(data_dique)

[========================================================================] 100%


In [20]:
dique_final.rename(columns=dict_dique_nombres).to_excel('_' + files[0][:20] + '.xls', index=False)

### Cruzar los números con las quejas generadas

In [31]:
import pandas as pd

datos_quejas_dique = pd.read_csv('DIQUE_QUEJAS.csv').drop_duplicates(subset='TELEFONO',keep='last')
datos_xml_dique = pd.read_excel('ML_DIQUE.xls').drop_duplicates(subset='SERVICE_ACCOUNT',keep='last').rename(columns={"SERVICE_ACCOUNT":"TELEFONO"})

In [52]:
datos_quejas_dique_xml = pd.merge(datos_quejas_dique,
                                  datos_xml_dique,
                                  on='TELEFONO',
                                  how='inner' )

In [69]:
datos_quejas_dique_xml.drop('XML', axis=1, inplace=True)

In [72]:
datos_quejas_dique_xml.to_csv('DIQUE_XML.csv', index=False)

### Seleccionamos las columnas que nos pueden ser útiles

In [17]:
import pandas as pd

datos = pd.read_csv('DIQUE_XML.csv')

In [23]:
lista = ["FOLIOQUEJA",
"TIPO_TAREA",
"TIPO_PRUEBA",
"CODRECEP",
"COPE",
"LIQUIDACION",
"DESC_ETAPA",
"nem_suppliercode",
"nem_bandWifi",
"nem_corrienteOptica",
"nem_edoAdministrativo",
"nem_edoOperativo",
"nem_esVdsl",
"nem_modTjta",
"nem_perfil",
"nem_senalRX",
"nem_senalTX",
"nem_severidad",
"nem_tecnologia",
"nem_temperatura",
"nem_vcic",
"nem_vlan",
"nem_vlanHsi",
"nem_vlanOnt",
"nem_voltajeOptico",
"nem_vpic",
"radius_code",
"radius_Activo",
"radius_VEL_DOWN_USR",
"radius_VEL_UP_USR",
"pe_code",
"sbc_code"]

In [25]:
datos[lista].to_csv('XML_PRUEBA.csv', index=False)

In [24]:
datos[lista].head(20)

,FOLIOQUEJA,TIPO_TAREA,TIPO_PRUEBA,CODRECEP,COPE,LIQUIDACION,DESC_ETAPA,nem_suppliercode,nem_bandWifi,...,nem_vlanHsi,nem_vlanOnt,nem_voltajeOptico,nem_vpic,radius_code,radius_Activo,radius_VEL_DOWN_USR,radius_VEL_UP_USR,pe_code,sbc_code
0,40651507,QI1LPZOD,PRUEBA DE DATOS,52PE,CTCLA,Conexión en caja distribución,PLANTA LIQUIDA,8025.0,Single Band,...,NaN,NaN,NaN,8.0,EXITO,S,10.0,1.0,NaN,NaN
1,40644077,QI1LPZOG,PRUEBA DE DATOS + VSI,74PEG,CTSF_,Reconstrucción Interior (bajante e interior),PLANTA LIQUIDA,8500.0,Dual Band,...,1409.0,881.0,3.22,0.0,EXITO,S,300.0,50.0,NaN,NaN
2,7024765,QI1LPZOD,PRUEBA DE DATOS,52PE,CTHEM,Filtros ADSL,PLANTA LIQUIDA,8025.0,Single Band,...,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN
3,17450388,QI1LPZOD,PRUEBA DE DATOS,52PE,CTTEN,Filtros ADSL,PLANTA LIQUIDA,8025.0,Single Band,...,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN
4,11919297,RI10PZOD,PRUEBA ELECTRICA,11,CTCHE,Instalación exterior +2 tramos,PLANTA LIQUIDA,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EXITO,NaN
5,11926345,RI1LPOED,PRUEBA DE DATOS,44TL,CTCON,Cambio de velocidad en el puerto,PLANTA LIQUIDA,8025.0,Dual Band,...,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN
6,19937719,QI2LPZOD,PRUEBA DE DATOS,52PE,CTACT,MODEM ADSL,PLANTA LIQUIDA,8025.0,Single Band,...,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN
7,40633056,QI1LPZOM,PRUEBA DE DATOS + VSI,74PE,CTTLH,Cambio de velocidad en el puerto,PLANTA LIQUIDA,8025.0,Dual Band,...,NaN,NaN,NaN,8.0,EXITO,S,16.0,1.0,NaN,NaN
8,40649472,QI1LPZOF,PRUEBA DATOS + ELECTRICA,52PF,CTMI_,MODEM óptico,PLANTA LIQUIDA,8500.0,Dual Band,...,1301.0,881.0,3.22,0.0,EXITO,S,20.0,3.0,NaN,NaN
9,40636785,QI1LPZOG,PRUEBA DE DATOS + VSI,74PEG,CTES_,Reconstrucción Interior (bajante e interior),PLANTA LIQUIDA,8500.0,Dual Band,...,1303.0,881.0,3.22,0.0,EXITO,S,20.0,3.0,NaN,NaN
